Load all libraries

In [1]:
# Loading libraries for the course
import random
import matplotlib.pyplot as plt             # Library used for plotting
from matplotlib.patches import Rectangle    # Module to plot a rectangle in the image
import urllib.request                       # Library to download data
import numpy as np                          # Library for array manipulation
import seaborn as sn                        # Library for advanced plotting
import pandas as pd                         # Library to manipulate data frames
import tifffile                             # Library to store numpy arrays as TIFF
import pathlib                              # Library to work with file paths
from pathlib import Path                    # Library to work with file paths
import skimage                              # Library for image manipulation. scikit-image
from skimage.io import imread               # Module from skimage
from matplotlib import animation            # Module to plot animations

from skimage import io, measure               # Module from skimage
from skimage.filters import gaussian # Module working with a gaussian filter                           # Library to work with file paths
import os
%matplotlib inline

from ipywidgets import interactive, HBox, VBox, Layout
import ipywidgets as widgets

from skimage.morphology import binary_dilation
from skimage.segmentation import watershed
from skimage.draw import polygon
from skimage.measure import regionprops
from skimage.color import label2rgb
from skimage.filters import threshold_otsu
from skimage.morphology import binary_erosion
from skimage.morphology import binary_closing
from skimage.morphology import binary_opening
from skimage.morphology import disk
from skimage.morphology import remove_small_objects
from scipy import ndimage as ndi              # Distance Transform
from skimage.feature import peak_local_max    # Local maxima in a matrix
from skimage.segmentation import watershed    # Watershed algorithm
from skimage.filters import difference_of_gaussians
import numpy as np
import matplotlib.pyplot as plt
from skimage import io as ski_io, color as ski_color, filters as ski_filters
from PIL import Image
import cellpose
import scipy as sp
import cv2

Metadata we want to get

In [2]:
metadata = {'time':[], 'cell_number':[], 'x':[], 'y':[],  'Alive':[], 
            'cell_size':[], 'nucleus_size':[], 'protein_concentration':[],
              'number_of_mRNA_in_nucleus':[], 'number_of_mRNA_in_cytoplasm':[], 
              'number_of_transcription_sites':[], 'intensity_of_transcription_sites':[]}

metadata = {'time':[], 'cell_number':[], 'x':[], 'y':[],  'Alive':[], 'nuclei_size':[],
            'cell_size':[],'protein_concentration':[]}

Function checking if center point of the cell is equal to 0 in the binarized red channel

In [3]:
# Function to check if the center points are 0 in the binarized image
def check_centers(image_array, centers):
    result = []
    for c in centers:
        x, y = c
        
        if image_array[x, y] == 1:
            result.append(1) #True
        else:
            result.append(0) #False
    return result

Big for loop to analize all .tif files in directory

In [4]:
#choose the directory

directory = Path('Drug1_1000nM/')
#directory = Path('Drug4_1000nM/')
#directory = Path('Drug10_1000nM/')

#list all the .tif files in the directory
files =directory.glob('*.tif')
# Pattern to match TIFF files

In [8]:
# Loop over all the files
for file in files:
    #metadata
    metadata = {'time':[], 'cell_number':[], 'x':[], 'y':[],  'Alive':[], 'nuclei_size':[],
            'cell_size':[],'protein_concentration':[]}
    
    #all variables that will be used in the loop
    cell_conc_all = []
    nuclei_areas_all = []
    result_all = []
    cell_areas_all = []
    

    # Read the image
    images = imread((file))

    #Get all the channels
    img_R = images[0,:,:,0]
    img_G = images[0,:,:,1]
    img_B = images[0,:,:,2]

    # Get the dimensions of the image
    [T,Y,X,C] = images.shape

    #take R channel and binarize it to get first data for metadata
    threshold_1 = 130
    img_R_bin = img_R > threshold_1
    img_R_bin = img_R_bin.astype(int)
    
    #using binarized image count number of cells 

    #labeling the cells
    img_R_label = measure.label(img_R_bin)

    #counting the number of cells
    num_cells = img_R_label.max()

    #finding centers of the cells
    props = measure.regionprops(img_R_label)

    centers = []
    for prop in props:
        centers.append(prop.centroid)
    centers = np.array(centers)

    #round centers locations to integers
    centers = np.round(centers).astype(int)

    # append metadata['frame_number'] with numbers 0 to T-1 num_cells times
    for i in range(num_cells):
        for j in range(T):
            metadata['time'].append(j)
            metadata['cell_number'].append(i+1)
            metadata['x'].append(centers[i,1])
            metadata['y'].append(centers[i,0])

    

    #for loop over all the images in the video

    for i in range(T):
        img_R = images[i,:,:,0]
        img_G = images[i,:,:,1]
        img_B = images[i,:,:,2]

        threshold = 130 
        img_bin = img_R > threshold
        img_bin = img_bin.astype(int)

        #check if centers we already have are still 0 in the binarized image
        # Check the centers
        result = check_centers(img_bin, centers)
        result_all.append(result) #result all is a list of lists, where each column is data for 1 cell in time
         
        #labeling the cells
        img_R_label = measure.label(img_bin)

        #counting the number of cells
        num_cells = img_R_label.max()

        #finding regions of the cells
        props = measure.regionprops(img_R_label)
        # get the areas of nuclei
        nuclei_areas = [region.area for region in props]

        
        #create a new vector with cell areas
        nuclei_areas2 = np.zeros(len(centers))
        
        for t in range(sum(np.array(result)==1)):
            nuclei_areas2[np.where(np.array(result)==1)[0][t]] = nuclei_areas[t]

        nuclei_areas_all.append(nuclei_areas2) #cell_areas_all is a list of lists, where each column is data for 1 cell in time


        #take G channel and binarize it to get second data for metadata
        #fileter on green chanell before binarization (gaussian filter)


        #gaussian filter
        img_G_filtered = ski_filters.gaussian(img_G, sigma=3)
        threshold_G = 0.225

        #binaryzation
        img_G_bin = img_G_filtered > threshold_G
        img_G_bin = img_G_bin.astype(int)

        #remove background noise
        # Define the kernel
        kernel = np.ones((4, 4), np.uint8)
        # Apply Erosion
        eroded = cv2.erode(img_G_filtered, kernel, iterations=2)
        # Apply Dilation
        dilated = cv2.dilate(eroded, kernel, iterations=2)

        mask_image = np.zeros(img_G.shape)
        mask_image[eroded>threshold_G] = 255
        new_mask = gaussian(mask_image, sigma=3)

        contours = measure.find_contours(new_mask, level=200)
        #contours_connected = np.vstack((contours))

        # downsample the contours
        #contours_connected = contours_connected[::20] 

        # make a new mask from the contours array
        watershed_starting_mask = np.zeros(img_G_filtered.shape).astype(int)                    # Pre-allocating an array with zeros. Notice the datatype.
        
        for contour in contours:
            rr, cc = polygon(contour[:,0], contour[:,1])           # Returns the coordinates inside the contour
            watershed_starting_mask[rr,cc] = 255                                           # Replacing all values inside the contour with ones.
            labels_G = ndi.label(watershed_starting_mask)

        sizes = []
        concentrations = []
         
        for val in range(len(centers)):
            pixel_val = labels_G[0][centers[val,0], centers[val,1]]

            if pixel_val == 0:
                sizes.append(0)
                concentrations.append(0)
                
            #count the number of pixels with the same value as the pixel in the center
            else:
                sizes.append(np.sum(labels_G[0] == pixel_val))
                concentrations.append(np.mean(img_G[labels_G[0] == pixel_val]))

        cell_areas_all.append(sizes)
        cell_conc_all.append(concentrations)




    #change result_all and cell_areas_all to arrays
    result_all = np.array(result_all)
    nuclei_areas_all = np.array(nuclei_areas_all)
    cell_areas_all = np.array(cell_areas_all)
    cell_conc_all = np.array(cell_conc_all)

    #append metadata['Alive'] and metadata['nuclei_size'] with the data from result_all and cell_areas_all
    for d in range(len(result)):
        for z in range(T):
            metadata['Alive'].append(result_all[z,d])
            metadata['nuclei_size'].append(nuclei_areas_all[z,d])
            metadata['protein_concentration'].append(cell_conc_all[z,d])
            metadata['cell_size'].append(cell_areas_all[z,d])
    

    #change metadata to dataframe
    metadata = pd.DataFrame(metadata)
    print(metadata)

    #save metadata to a csv file in the same directory as the images
    file_name = file.stem + '_metadata.csv'
    file_name = 'Drug1_1000nM/' + file_name
    #file_name = 'Drug4_1000nM/' + file_name
    #file_name = 'Drug10_1000nM/' + file_name
    metadata.to_csv(file_name, index=False)

    





     time  cell_number    x    y  Alive  nuclei_size  cell_size  \
0       0            1  259  101      1       2433.0       6079   
1       1            1  259  101      1       2438.0       6133   
2       2            1  259  101      1       2445.0       6165   
3       3            1  259  101      1       2429.0       6146   
4       4            1  259  101      1       2451.0       6051   
..    ...          ...  ...  ...    ...          ...        ...   
247    16           12  426  596      1       2641.0       6640   
248    17           12  426  596      1       2658.0       6760   
249    18           12  426  596      0          0.0          0   
250    19           12  426  596      0          0.0          0   
251    20           12  426  596      0          0.0          0   

     protein_concentration  
0               120.008883  
1               126.417251  
2               131.295053  
3               132.242271  
4               124.523715  
..                   